In [1]:
import os
import subprocess
from pathlib import Path

def compress_audio_files():
    # Define the songs folder path (adjust this to your actual path)
    songs_folder = Path("./songs")  # Change this to your songs folder path
    
    # Check if songs folder exists
    if not songs_folder.exists():
        print(f"Songs folder not found at: {songs_folder}")
        return
    
    # Get the first subfolder in the songs directory
    subfolders = [f for f in songs_folder.iterdir() if f.is_dir()]
    
    if not subfolders:
        print("No subfolders found in the songs directory")
        return
    
    # Sort to ensure consistent "first" folder selection
    first_folder = sorted(subfolders)[0]
    print(f"Processing folder: {first_folder.name}")
    
    # Create output folder
    output_folder = songs_folder / f"{first_folder.name}_compressed"
    output_folder.mkdir(exist_ok=True)
    
    # Supported audio formats
    audio_extensions = {'.mp3', '.webm', '.wav', '.m4a', '.flac', '.ogg'}
    
    # Find all audio files in the first folder
    audio_files = []
    for ext in audio_extensions:
        audio_files.extend(first_folder.glob(f"*{ext}"))
    
    if not audio_files:
        print(f"No audio files found in {first_folder.name}")
        return
    
    print(f"Found {len(audio_files)} audio files to compress")
    
    # Process each audio file
    for i, audio_file in enumerate(audio_files, 1):
        try:
            # Create output filename (convert everything to mp3 for consistency)
            output_file = output_folder / f"{audio_file.stem}_compressed.mp3"
            
            print(f"[{i}/{len(audio_files)}] Compressing: {audio_file.name}")
            
            # FFmpeg command for compression
            # -b:a 128k sets audio bitrate to 128kbps (good quality but smaller size)
            # -ar 44100 sets sample rate to 44.1kHz
            # -ac 2 sets to stereo
            cmd = [
                'ffmpeg',
                '-i', str(audio_file),
                '-b:a', '128k',  # Audio bitrate (lower = smaller file)
                '-ar', '44100',  # Sample rate
                '-ac', '2',      # Stereo channels
                '-y',            # Overwrite output file if exists
                str(output_file)
            ]
            
            # Run FFmpeg with suppressed output
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            if result.returncode == 0:
                # Get file sizes for comparison
                original_size = audio_file.stat().st_size
                compressed_size = output_file.stat().st_size
                compression_ratio = (1 - compressed_size/original_size) * 100
                
                print(f"  ✓ Success! Reduced size by {compression_ratio:.1f}%")
            else:
                print(f"  ✗ Failed to compress {audio_file.name}")
                print(f"  Error: {result.stderr}")
                
        except Exception as e:
            print(f"  ✗ Error processing {audio_file.name}: {str(e)}")
    
    print(f"\nCompression complete! Files saved to: {output_folder}")

def check_ffmpeg():
    """Check if FFmpeg is installed"""
    try:
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if __name__ == "__main__":
    # Check if FFmpeg is available
    if not check_ffmpeg():
        print("Error: FFmpeg is not installed or not in PATH")
        print("Please install FFmpeg:")
        print("- Windows: Download from https://ffmpeg.org/download.html")
        print("- macOS: brew install ffmpeg")
        print("- Linux: sudo apt install ffmpeg (Ubuntu/Debian)")
        exit(1)
    
    print("Audio File Compressor")
    print("=" * 30)
    
    # Update the songs folder path if needed
    songs_path = input("Enter your songs folder path (or press Enter for './songs'): ").strip()
    if songs_path:
        # Update the path in the function
        print(f"Using custom path: {songs_path}")
    
    compress_audio_files()

Audio File Compressor
Processing folder: All in one
Found 433 audio files to compress
[1/433] Compressing: Aaromale-AR-Rahman-Alphons-Joseph.mp3
  ✓ Success! Reduced size by 33.3%
[2/433] Compressing: Aasai-Aasai-Shankar-Mahadevan-Sujatha-Kabilan.mp3
  ✓ Success! Reduced size by 33.3%
[3/433] Compressing: Aasai-Oru-Pulveli-Santhosh-Narayanan-Pradeep-Kumar-Kalyani-Nair.mp3
  ✓ Success! Reduced size by 33.3%
[4/433] Compressing: Aathadi-Aathadi-Harris-Jayaraj-Bhavatharini-Tippu-Dhanush-Abhay-Jodhpurkar.mp3
  ✓ Success! Reduced size by 33.3%
[5/433] Compressing: Abinaya-Mugen-Rao.mp3
  ✓ Success! Reduced size by 33.3%
[6/433] Compressing: Adi-Penne-Duet-From-Naam-Series-Stephen-Zechariah-T-Suriavelan-Srinisha-Jayaseelan.mp3


KeyboardInterrupt: 